In [1]:
#PACKAGES
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.tsa.api import SimpleExpSmoothing, Holt, ExponentialSmoothing
import requests
import json
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Define the API endpoint
api_url = "http://172.16.1.59:8080/v1/web/test12"

# Fetch data from the API
response = requests.get(api_url)

# Check if the request was successful
if response.status_code == 200:
    # Convert the JSON response to a Python dictionary
    data = response.json()
    
    # Create a pandas DataFrame from the data
    # Assuming the API response is a list of dictionaries
    df = pd.DataFrame(data['data'])
    
else:
    print(f"Failed to fetch data: {response.status_code}")
    
display(df)
#pd.options.display.max_colwidth = 100

#link clone repository
#https://github.com/doctermath/hnf01b.git

ConnectTimeout: HTTPConnectionPool(host='172.16.1.59', port=8080): Max retries exceeded with url: /v1/web/test12 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x00000267AD186C90>, 'Connection to 172.16.1.59 timed out. (connect timeout=None)'))

In [3]:
file_path = "data/datadummy.json"
with open(file_path,'r') as f:
    json_data = json.load(f)
data = pd.DataFrame(json_data)

data = pd.DataFrame(json_data['data'])
print(data)

       period branch agency partno  \
0  2025-01-01    999    99P   9901   
1  2025-01-01    999    99P   9902   
2  2025-01-01    999    99Q   9903   
3  2025-01-01    888    88A   8804   
4  2025-01-01    888    88A   8805   
5  2025-01-01    999    99Q   9906   
6  2025-01-01    888    88B   8807   
7  2025-01-01    999    99Q   9908   
8  2025-01-01    999    99R   9909   
9  2025-01-01    888    88B   8810   

                                                   D  
0  [100, 105, 110, 108, 115, 120, 125, 130, 128, ...  
1  [200, 210, 220, 215, 230, 240, 245, 250, 255, ...  
2  [50, 55, 60, 58, 65, 70, 75, 80, 85, 88, 90, 9...  
3  [300, 310, 320, 315, 330, 340, 345, 350, 360, ...  
4  [500, 505, 510, 515, 520, 525, 530, 535, 540, ...  
5  [150, 155, 160, 158, 165, 170, 175, 180, 185, ...  
6  [400, 405, 410, 408, 415, 420, 425, 430, 435, ...  
7  [80, 85, 90, 88, 95, 100, 105, 110, 115, 120, ...  
8  [250, 260, 270, 265, 280, 290, 295, 300, 310, ...  
9  [600, 610, 620, 615, 630, 64

In [4]:
#PARAMETER
#WMA
weights = [0.1, 0.3, 0.6]

#EWMA
alpha_wma = 0.4

#SES & DES
alpha_ses = 0.65  # ubah nilai alpha (semakin besar semakin berat ke data terbaru)
beta_des = 0.45   # ubah nilai beta (semakin besar semakin cepat beradaptasi, kalo rendah bisa terjadi lag)

In [5]:
# get mean and standart deviation of first 12 data
data['mean_12'] = data['D'].apply(lambda x: np.mean(x[:12]))
data['std_12'] = data['D'].apply(lambda x: np.std(x[:12]))

# get upper bound from mean and std
data['ub'] = data['mean_12'] + 1.5 * data['std_12']

# limit the original data to upper bound
data['clipped_d'] = data.apply(lambda row: np.clip(row['D'][:12], 0, row['ub']).tolist(), axis=1)
display(data)

,period,branch,agency,partno,D,mean_12,std_12,ub,clipped_d
0,2025-01-01,999,99P,9901,"[100, 105, 110, 108, 115, 120, 125, 130, 128, ...",121.750000,13.833082,142.499623,"[100.0, 105.0, 110.0, 108.0, 115.0, 120.0, 125..."
1,2025-01-01,999,99P,9902,"[200, 210, 220, 215, 230, 240, 245, 250, 255, ...",238.333333,22.110832,271.499581,"[200.0, 210.0, 220.0, 215.0, 230.0, 240.0, 245..."
2,2025-01-01,999,99Q,9903,"[50, 55, 60, 58, 65, 70, 75, 80, 85, 88, 90, 9...",72.333333,14.102797,93.487529,"[50.0, 55.0, 60.0, 58.0, 65.0, 70.0, 75.0, 80...."
3,2025-01-01,888,88A,8804,"[300, 310, 320, 315, 330, 340, 345, 350, 360, ...",340.000000,23.979158,375.968736,"[300.0, 310.0, 320.0, 315.0, 330.0, 340.0, 345..."
4,2025-01-01,888,88A,8805,"[500, 505, 510, 515, 520, 525, 530, 535, 540, ...",527.500000,17.260263,553.390394,"[500.0, 505.0, 510.0, 515.0, 520.0, 525.0, 530..."
5,2025-01-01,999,99Q,9906,"[150, 155, 160, 158, 165, 170, 175, 180, 185, ...",173.583333,15.850648,197.359306,"[150.0, 155.0, 160.0, 158.0, 165.0, 170.0, 175..."
6,2025-01-01,888,88B,8807,"[400, 405, 410, 408, 415, 420, 425, 430, 435, ...",423.583333,15.850648,447.359306,"[400.0, 405.0, 410.0, 408.0, 415.0, 420.0, 425..."
7,2025-01-01,999,99Q,9908,"[80, 85, 90, 88, 95, 100, 105, 110, 115, 120, ...",103.583333,15.850648,127.359306,"[80.0, 85.0, 90.0, 88.0, 95.0, 100.0, 105.0, 1..."
8,2025-01-01,999,99R,9909,"[250, 260, 270, 265, 280, 290, 295, 300, 310, ...",290.000000,23.979158,325.968736,"[250.0, 260.0, 270.0, 265.0, 280.0, 290.0, 295..."
9,2025-01-01,888,88B,8810,"[600, 610, 620, 615, 630, 640, 645, 650, 660, ...",640.000000,23.979158,675.968736,"[600.0, 610.0, 620.0, 615.0, 630.0, 640.0, 645..."


In [8]:
# Calculate Simple Moving Average
def calculate_ma(list):
    oldData = []
    newData = []
    for i in list:
        # store calculated data to old list
        oldData.append(i)
        newData.append(np.mean(oldData))
    return newData

data['ma'] = data['clipped_d'].apply(calculate_ma)
data['ma_result'] = data['ma'].apply(lambda x: x[-1:])
data['ma_result'] = data['clipped_d'].apply(lambda x: np.mean(x))
display(data)


,period,branch,agency,partno,D,mean_12,std_12,ub,clipped_d,ma,ma_result
0,2025-01-01,999,99P,9901,"[100, 105, 110, 108, 115, 120, 125, 130, 128, ...",121.750000,13.833082,142.499623,"[100.0, 105.0, 110.0, 108.0, 115.0, 120.0, 125...","[100.0, 102.5, 105.0, 105.75, 107.6, 109.66666...",121.541635
1,2025-01-01,999,99P,9902,"[200, 210, 220, 215, 230, 240, 245, 250, 255, ...",238.333333,22.110832,271.499581,"[200.0, 210.0, 220.0, 215.0, 230.0, 240.0, 245...","[200.0, 205.0, 210.0, 211.25, 215.0, 219.16666...",238.333333
2,2025-01-01,999,99Q,9903,"[50, 55, 60, 58, 65, 70, 75, 80, 85, 88, 90, 9...",72.333333,14.102797,93.487529,"[50.0, 55.0, 60.0, 58.0, 65.0, 70.0, 75.0, 80....","[50.0, 52.5, 55.0, 55.75, 57.6, 59.66666666666...",72.333333
3,2025-01-01,888,88A,8804,"[300, 310, 320, 315, 330, 340, 345, 350, 360, ...",340.000000,23.979158,375.968736,"[300.0, 310.0, 320.0, 315.0, 330.0, 340.0, 345...","[300.0, 305.0, 310.0, 311.25, 315.0, 319.16666...",340.000000
4,2025-01-01,888,88A,8805,"[500, 505, 510, 515, 520, 525, 530, 535, 540, ...",527.500000,17.260263,553.390394,"[500.0, 505.0, 510.0, 515.0, 520.0, 525.0, 530...","[500.0, 502.5, 505.0, 507.5, 510.0, 512.5, 515...",527.365866
5,2025-01-01,999,99Q,9906,"[150, 155, 160, 158, 165, 170, 175, 180, 185, ...",173.583333,15.850648,197.359306,"[150.0, 155.0, 160.0, 158.0, 165.0, 170.0, 175...","[150.0, 152.5, 155.0, 155.75, 157.6, 159.66666...",173.363275
6,2025-01-01,888,88B,8807,"[400, 405, 410, 408, 415, 420, 425, 430, 435, ...",423.583333,15.850648,447.359306,"[400.0, 405.0, 410.0, 408.0, 415.0, 420.0, 425...","[400.0, 402.5, 405.0, 405.75, 407.6, 409.66666...",423.363275
7,2025-01-01,999,99Q,9908,"[80, 85, 90, 88, 95, 100, 105, 110, 115, 120, ...",103.583333,15.850648,127.359306,"[80.0, 85.0, 90.0, 88.0, 95.0, 100.0, 105.0, 1...","[80.0, 82.5, 85.0, 85.75, 87.6, 89.66666666666...",103.363275
8,2025-01-01,999,99R,9909,"[250, 260, 270, 265, 280, 290, 295, 300, 310, ...",290.000000,23.979158,325.968736,"[250.0, 260.0, 270.0, 265.0, 280.0, 290.0, 295...","[250.0, 255.0, 260.0, 261.25, 265.0, 269.16666...",290.000000
9,2025-01-01,888,88B,8810,"[600, 610, 620, 615, 630, 640, 645, 650, 660, ...",640.000000,23.979158,675.968736,"[600.0, 610.0, 620.0, 615.0, 630.0, 640.0, 645...","[600.0, 605.0, 610.0, 611.25, 615.0, 619.16666...",640.000000


In [ ]:
# Calculate Exponential Weighted Moving Average (EWMA)
def ewma(list, alpha):
    df = pd.DataFrame(list)
    df['ewma'] = df.ewm(alpha=alpha, adjust=False).mean()
    return df['ewma'].tolist()

def ewma_forecast(list, alpha):
    ewma_values = ewma(list, alpha)
    if len(ewma_values) > 0:
        # Prediction for the next period
        next_forecast = (1 - alpha) * ewma_values[-1]
    else:
        next_forecast = None
    return ewma_values, next_forecast

ewma_alpha = 0.4
data['ewma'], data['ewma_forecast'] = zip(*data['clipped_d'].apply(lambda x: ewma_forecast(x, ewma_alpha)))

display(data)

,period,branch,agency,partno,D,mean_12,std_12,ub,clipped_d,ma,ma_result,ewma,ewma_result,ewma_forecast
0,2025-01-01,999,99P,9901,"[100, 105, 110, 108, 115, 120, 125, 130, 128, ...",121.750000,13.833082,142.499623,"[100.0, 105.0, 110.0, 108.0, 115.0, 120.0, 125...","[100.0, 102.5, 105.0, 105.75, 107.6, 109.66666...",121.541635,"[100.0, 102.0, 105.19999999999999, 106.32, 109...",[137.039210727424],82.223526
1,2025-01-01,999,99P,9902,"[200, 210, 220, 215, 230, 240, 245, 250, 255, ...",238.333333,22.110832,271.499581,"[200.0, 210.0, 220.0, 215.0, 230.0, 240.0, 245...","[200.0, 205.0, 210.0, 211.25, 215.0, 219.16666...",238.333333,"[200.0, 204.0, 210.39999999999998, 212.2399999...",[262.27168419839995],157.363011
2,2025-01-01,999,99Q,9903,"[50, 55, 60, 58, 65, 70, 75, 80, 85, 88, 90, 9...",72.333333,14.102797,93.487529,"[50.0, 55.0, 60.0, 58.0, 65.0, 70.0, 75.0, 80....","[50.0, 52.5, 55.0, 55.75, 57.6, 59.66666666666...",72.333333,"[50.0, 52.0, 55.2, 56.32, 59.792, 63.8752, 68....",[87.8761613312],52.725697
3,2025-01-01,888,88A,8804,"[300, 310, 320, 315, 330, 340, 345, 350, 360, ...",340.000000,23.979158,375.968736,"[300.0, 310.0, 320.0, 315.0, 330.0, 340.0, 345...","[300.0, 305.0, 310.0, 311.25, 315.0, 319.16666...",340.000000,"[300.0, 304.0, 310.4, 312.24, 319.344, 327.606...",[366.6236841983999],219.974211
4,2025-01-01,888,88A,8805,"[500, 505, 510, 515, 520, 525, 530, 535, 540, ...",527.500000,17.260263,553.390394,"[500.0, 505.0, 510.0, 515.0, 520.0, 525.0, 530...","[500.0, 502.5, 505.0, 507.5, 510.0, 512.5, 515...",527.365866,"[500.0, 502.0, 505.2, 509.12, 513.472, 518.083...",[546.8833673678039],328.130020
5,2025-01-01,999,99Q,9906,"[150, 155, 160, 158, 165, 170, 175, 180, 185, ...",173.583333,15.850648,197.359306,"[150.0, 155.0, 160.0, 158.0, 165.0, 170.0, 175...","[150.0, 152.5, 155.0, 155.75, 157.6, 159.66666...",173.363275,"[150.0, 152.0, 155.2, 156.32, 159.791999999999...",[191.50788372071816],114.904730
6,2025-01-01,888,88B,8807,"[400, 405, 410, 408, 415, 420, 425, 430, 435, ...",423.583333,15.850648,447.359306,"[400.0, 405.0, 410.0, 408.0, 415.0, 420.0, 425...","[400.0, 402.5, 405.0, 405.75, 407.6, 409.66666...",423.363275,"[400.0, 402.0, 405.2, 406.32, 409.792, 413.875...",[441.5078837207181],264.904730
7,2025-01-01,999,99Q,9908,"[80, 85, 90, 88, 95, 100, 105, 110, 115, 120, ...",103.583333,15.850648,127.359306,"[80.0, 85.0, 90.0, 88.0, 95.0, 100.0, 105.0, 1...","[80.0, 82.5, 85.0, 85.75, 87.6, 89.66666666666...",103.363275,"[80.0, 82.0, 85.19999999999999, 86.32, 89.792,...",[121.50788372071816],72.904730
8,2025-01-01,999,99R,9909,"[250, 260, 270, 265, 280, 290, 295, 300, 310, ...",290.000000,23.979158,325.968736,"[250.0, 260.0, 270.0, 265.0, 280.0, 290.0, 295...","[250.0, 255.0, 260.0, 261.25, 265.0, 269.16666...",290.000000,"[250.0, 254.0, 260.4, 262.24, 269.344, 277.606...",[316.6236841983999],189.974211
9,2025-01-01,888,88B,8810,"[600, 610, 620, 615, 630, 640, 645, 650, 660, ...",640.000000,23.979158,675.968736,"[600.0, 610.0, 620.0, 615.0, 630.0, 640.0, 645...","[600.0, 605.0, 610.0, 611.25, 615.0, 619.16666...",640.000000,"[600.0, 604.0, 610.4, 612.24, 619.344, 627.606...",[666.6236841983998],399.974211


In [15]:
#LINEAR REGRESSION
#  Calculate Linear Regression
def lr(x):
    df = pd.DataFrame()
    df['y'] = x
    df['x'] = range(1, len(df) + 1)
    model =  LinearRegression()
    model.fit(df[['x']], df['y'])
    df.loc[len(df), 'x'] = len(df) + 1
    return model.predict(df[['x']])

data['lr'] = data['clipped_d'].apply(lambda x: lr(x))
data['lr_result'] = data['lr'].apply(lambda x: x[-1:])
display(data)


,period,branch,agency,partno,D,mean_12,std_12,ub,clipped_d,ma,ma_result,ewma,ewma_result,ewma_forecast,lr,lr_result
0,2025-01-01,999,99P,9901,"[100, 105, 110, 108, 115, 120, 125, 130, 128, ...",121.750000,13.833082,142.499623,"[100.0, 105.0, 110.0, 108.0, 115.0, 120.0, 125...","[100.0, 102.5, 105.0, 105.75, 107.6, 109.66666...",121.541635,"[100.0, 102.0, 105.19999999999999, 106.32, 109...",[137.039210727424],82.223526,"[100.24363801403076, 104.11600115527608, 107.9...",[146.71199570897454]
1,2025-01-01,999,99P,9902,"[200, 210, 220, 215, 230, 240, 245, 250, 255, ...",238.333333,22.110832,271.499581,"[200.0, 210.0, 220.0, 215.0, 230.0, 240.0, 245...","[200.0, 205.0, 210.0, 211.25, 215.0, 219.16666...",238.333333,"[200.0, 204.0, 210.39999999999998, 212.2399999...",[262.27168419839995],157.363011,"[203.52564102564105, 209.85431235431236, 216.1...",[279.46969696969694]
2,2025-01-01,999,99Q,9903,"[50, 55, 60, 58, 65, 70, 75, 80, 85, 88, 90, 9...",72.333333,14.102797,93.487529,"[50.0, 55.0, 60.0, 58.0, 65.0, 70.0, 75.0, 80....","[50.0, 52.5, 55.0, 55.75, 57.6, 59.66666666666...",72.333333,"[50.0, 52.0, 55.2, 56.32, 59.792, 63.8752, 68....",[87.8761613312],52.725697,"[50.06410256410256, 54.11305361305361, 58.1620...",[98.65151515151513]
3,2025-01-01,888,88A,8804,"[300, 310, 320, 315, 330, 340, 345, 350, 360, ...",340.000000,23.979158,375.968736,"[300.0, 310.0, 320.0, 315.0, 330.0, 340.0, 345...","[300.0, 305.0, 310.0, 311.25, 315.0, 319.16666...",340.000000,"[300.0, 304.0, 310.4, 312.24, 319.344, 327.606...",[366.6236841983999],219.974211,"[302.11538461538464, 309.00349650349654, 315.8...",[384.77272727272725]
4,2025-01-01,888,88A,8805,"[500, 505, 510, 515, 520, 525, 530, 535, 540, ...",527.500000,17.260263,553.390394,"[500.0, 505.0, 510.0, 515.0, 520.0, 525.0, 530...","[500.0, 502.5, 505.0, 507.5, 510.0, 512.5, 515...",527.365866,"[500.0, 502.0, 505.2, 509.12, 513.472, 518.083...",[546.8833673678039],328.130020,"[500.2063597472423, 505.14445182306963, 510.08...",[559.46346465717]
5,2025-01-01,999,99Q,9906,"[150, 155, 160, 158, 165, 170, 175, 180, 185, ...",173.583333,15.850648,197.359306,"[150.0, 155.0, 160.0, 158.0, 165.0, 170.0, 175...","[150.0, 152.5, 155.0, 155.75, 157.6, 159.66666...",173.363275,"[150.0, 152.0, 155.2, 156.32, 159.791999999999...",[191.50788372071816],114.904730,"[148.82573000335958, 153.28710191144262, 157.7...",[202.36219290035604]
6,2025-01-01,888,88B,8807,"[400, 405, 410, 408, 415, 420, 425, 430, 435, ...",423.583333,15.850648,447.359306,"[400.0, 405.0, 410.0, 408.0, 415.0, 420.0, 425...","[400.0, 402.5, 405.0, 405.75, 407.6, 409.66666...",423.363275,"[400.0, 402.0, 405.2, 406.32, 409.792, 413.875...",[441.5078837207181],264.904730,"[398.8257300033595, 403.28710191144256, 407.74...",[452.362192900356]
7,2025-01-01,999,99Q,9908,"[80, 85, 90, 88, 95, 100, 105, 110, 115, 120, ...",103.583333,15.850648,127.359306,"[80.0, 85.0, 90.0, 88.0, 95.0, 100.0, 105.0, 1...","[80.0, 82.5, 85.0, 85.75, 87.6, 89.66666666666...",103.363275,"[80.0, 82.0, 85.19999999999999, 86.32, 89.792,...",[121.50788372071816],72.904730,"[78.82573000335958, 83.28710191144262, 87.7484...",[132.36219290035604]
8,2025-01-01,999,99R,9909,"[250, 260, 270, 265, 280, 290, 295, 300, 310, ...",290.000000,23.979158,325.968736,"[250.0, 260.0, 270.0, 265.0, 280.0, 290.0, 295...","[250.0, 255.0, 260.0, 261.25, 265.0, 269.16666...",290.000000,"[250.0, 254.0, 260.4, 262.24, 269.344, 277.606...",[316.6236841983999],189.974211,"[252.11538461538464, 259.00349650349654, 265.8...",[334.77272727272725]
9,2025-01-01,888,88B,8810,"[600, 610, 620, 615, 630, 640, 645, 650, 660, ...",640.000000,23.979158,675.968736,"[600.0, 610.0, 620.0, 615.0, 630.0, 640.0, 645...","[600.0, 605.0, 610.0, 611.25, 615.0, 619.16666...",640.000000,"[600.0, 604.0, 610.4, 612.24, 619.344, 627.606...",[666.6236841983998],399.974211,"[602.1153846153846, 609.0034965034965, 615.891...",[684.7727272727273]


In [ ]:
#POLYNOMIAL 2ND AND 3RD
# Calculate Polynomial Regression
def pr(x, pr_degree):
    df = pd.DataFrame()
    df['y'] = x
    df['x'] = range(1, len(df) + 1)

    X = df[['x']]  # Independent variable (reshape to 2D array)
    y = df['y']    # Dependent variable

    poly = PolynomialFeatures(degree=pr_degree)  # Create polynomial features
    X_poly = poly.fit_transform(X)  # Transform input features
    poly_model = LinearRegression()  # Initialize linear regression model
    poly_model.fit(X_poly, y)  # Fit polynomial model

    df.loc[len(df), 'x'] = len(df) + 1
    X_all_poly = poly.transform(df[['x']])
    return poly_model.predict(X_all_poly)  

data['pr2'] = data['clipped_d'].apply(lambda x: pr(x, 2))
data['pr2_result'] = data['pr2'].apply(lambda x: x[-1:])
data['pr3'] = data['clipped_d'].apply(lambda x: pr(x, 3))
data['pr3_result'] = data['pr3'].apply(lambda x: x[-1:])
display(data)


In [ ]:
#SES
def ses(x, alpha):
    df = pd.DataFrame()
    df['y'] = x
    df['x'] = range(1, len(df) + 1)
    df.loc[len(df), 'x'] = len(df) + 1

    new_data = SimpleExpSmoothing(df['y']).fit(smoothing_level=alpha, optimized=False).fittedvalues
    return new_data.tolist()

data['ses'] = data['clipped_d'].apply(lambda x: ses(x, 0.65))
data['ses_result'] = data['ses'].apply(lambda x: x[-1:])
display(data)

In [ ]:
#DES
def des(x, alpha, beta):
    df = pd.DataFrame()
    df['y'] = x
    df['x'] = range(1, len(df) + 1)
    df.loc[len(df), 'x'] = len(df) + 1

    new_data = ExponentialSmoothing(df['y'], trend='add', seasonal=None).fit(smoothing_level=alpha, smoothing_trend=beta, optimized=False).fittedvalues
    return new_data.tolist()

data['des'] = data['clipped_d'].apply(lambda x: des(x, 0.65, 0.45))
data['des_result'] = data['des'].apply(lambda x: x[-1:])
display(data)

In [ ]:
# calculate R2 Score and RMSE

def metric(x):
    period_lenght = len(x['clipped_d'])
    df = pd.DataFrame()
    df['period'] = range(1, period_lenght + 1)
    df['qty'] = x['clipped_d'][:period_lenght]
    df['ma'] = x['ma'][:period_lenght]
    df['ewma'] = x['ewma'][:period_lenght]
    df['lr'] = x['lr'][:period_lenght]
    df['pr2'] = x['pr2'][:period_lenght]
    df['pr3'] = x['pr3'][:period_lenght]
    df['ses'] = x['ses'][:period_lenght]
    df['des'] = x['des'][:period_lenght]
    # display(df)
    
    # df = pd.concat([pd.DataFrame(x['lr']), df], axis=1)
    result = []
    for i in df.iloc[:, -7:]:
        rmse = np.sqrt(mean_squared_error(df['qty'], df[i]))  # Calculate RMSE
        r2 = r2_score(df['qty'], df[i])  # Calculate R2
        mae = mean_absolute_error(df['qty'], df[i])
        result.append({'model': i, 'RMSE': rmse, 'MAE': mae, 'R2': r2})
        
    # display(result)
    # df_result = pd.DataFrame()

    
    return result


data['metric'] = data.apply(lambda x: metric(x), axis=1)

# display(data['metric'][1])
display(data)

In [32]:
# BEST MODEL SELECTION
def best_select(x, key):
    return max(x, key=lambda x: x['R2'])[key]

def best_number(x):
    return x[x['best_model']][-1]

data['best_model'] = data['metric'].apply(lambda x: best_select(x, 'model'))
data['best_r2'] = data['metric'].apply(lambda x: best_select(x, 'R2'))
data['best_value'] = data.apply(lambda x: best_number(x), axis=1)
data['FD'] = np.ceil(data['best_value'])
display(data)

,period,branch,agency,partno,D,mean_12,std_12,ub,clipped_d,ma,...,pr3_result,ses,ses_result,des,des_result,metric,best_model,best_r2,best_value,FD
0,2025-01-01,999,99P,9901,"[100, 105, 110, 108, 115, 120, 125, 130, 128, ...",121.750000,13.833082,142.499623,"[100.0, 105.0, 110.0, 108.0, 115.0, 120.0, 125...","[100.0, 102.5, 105.0, 105.75, 107.6, 109.66666...",...,[145.98939697731214],"[100.0, 100.0, 103.25, 107.63749999999999, 107...",[140.70186865495282],"[100.36363636363632, 103.85121212121213, 108.6...",[146.6576510767954],"[{'model': 'ma', 'RMSE': 12.686563308919864, '...",pr3,0.981268,145.989397,146.0
1,2025-01-01,999,99P,9902,"[200, 210, 220, 215, 230, 240, 245, 250, 255, ...",238.333333,22.110832,271.499581,"[200.0, 210.0, 220.0, 215.0, 230.0, 240.0, 245...","[200.0, 205.0, 210.0, 211.25, 215.0, 219.16666...",...,[272.3737373737367],"[200.0, 200.0, 206.5, 215.27499999999998, 215....",[267.3028514159022],"[202.3636363636363, 206.83287878787883, 215.82...",[275.0959026789569],"[{'model': 'ma', 'RMSE': 20.691195756599676, '...",pr3,0.983232,272.373737,273.0
2,2025-01-01,999,99Q,9903,"[50, 55, 60, 58, 65, 70, 75, 80, 85, 88, 90, 9...",72.333333,14.102797,93.487529,"[50.0, 55.0, 60.0, 58.0, 65.0, 70.0, 75.0, 80....","[50.0, 52.5, 55.0, 55.75, 57.6, 59.66666666666...",...,[92.34343434343444],"[50.0, 50.0, 53.25, 57.6375, 57.873125, 62.505...",[90.8110696360888],"[49.29090909090907, 54.250136363636365, 59.455...",[95.71386142084489],"[{'model': 'ma', 'RMSE': 13.255584845803016, '...",pr3,0.990261,92.343434,93.0
3,2025-01-01,888,88A,8804,"[300, 310, 320, 315, 330, 340, 345, 350, 360, ...",340.000000,23.979158,375.968736,"[300.0, 310.0, 320.0, 315.0, 330.0, 340.0, 345...","[300.0, 305.0, 310.0, 311.25, 315.0, 319.16666...",...,[378.68686868686837],"[300.0, 300.0, 306.5, 315.275, 315.09625, 324....",[372.2278201659022],"[301.1818181818181, 307.2497727272727, 316.677...",[381.30694454194224],"[{'model': 'ma', 'RMSE': 22.492113354977707, '...",pr3,0.985883,378.686869,379.0
4,2025-01-01,888,88A,8805,"[500, 505, 510, 515, 520, 525, 530, 535, 540, ...",527.500000,17.260263,553.390394,"[500.0, 505.0, 510.0, 515.0, 520.0, 525.0, 530...","[500.0, 502.5, 505.0, 507.5, 510.0, 512.5, 515...",...,[557.8538586286794],"[500.0, 500.0, 503.25, 507.6375, 512.423125, 5...",[551.2614743831778],"[499.99999999999966, 505.0, 510.0000000000001,...",[558.4829463181482],"[{'model': 'ma', 'RMSE': 16.02998764708569, 'M...",pr3,0.999820,557.853859,558.0
5,2025-01-01,999,99Q,9906,"[150, 155, 160, 158, 165, 170, 175, 180, 185, ...",173.583333,15.850648,197.359306,"[150.0, 155.0, 160.0, 158.0, 165.0, 170.0, 175...","[150.0, 152.5, 155.0, 155.75, 157.6, 159.66666...",...,[201.36796352061617],"[150.0, 150.0, 153.25, 157.6375, 157.873125, 1...",[195.59136851905578],"[148.99999999999991, 154.3425, 159.65469375000...",[202.5704744882856],"[{'model': 'ma', 'RMSE': 14.60440716119259, 'M...",pr3,0.992396,201.367964,202.0
6,2025-01-01,888,88B,8807,"[400, 405, 410, 408, 415, 420, 425, 430, 435, ...",423.583333,15.850648,447.359306,"[400.0, 405.0, 410.0, 408.0, 415.0, 420.0, 425...","[400.0, 402.5, 405.0, 405.75, 407.6, 409.66666...",...,[451.36796352061583],"[400.0, 400.0, 403.25, 407.6375, 407.873124999...",[445.5913685190558],"[398.99999999999983, 404.3425, 409.65469375000...",[452.5704744882856],"[{'model': 'ma', 'RMSE': 14.604407161192592, '...",pr3,0.992396,451.367964,452.0
7,2025-01-01,999,99Q,9908,"[80, 85, 90, 88, 95, 100, 105, 110, 115, 120, ...",103.583333,15.850648,127.359306,"[80.0, 85.0, 90.0, 88.0, 95.0, 100.0, 105.0, 1...","[80.0, 82.5, 85.0, 85.75, 87.6, 89.66666666666...",...,[131.36796352061617],"[80.0, 80.0, 83.25, 87.6375, 87.873125, 92.505...",[125.59136851905578],"[78.99999999999997, 84.3425, 89.65469375, 94.8...",[132.5704744882856],"[{'model': 'ma', 'RMSE': 14.604407161192592, '...",pr3,0.992396,131.367964,132.0
8,2025-01-01,999,99R,9909,"[250, 260, 270, 265, 280, 290, 295, 300, 310, ...",290.000000,23.979158,325.968736,"[250.0, 260.0, 270.0, 265.